In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer, MICEImputer
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**1. Loading the data**

In [2]:
application_train=pd.read_csv('../input/application_train.csv')
bureau=pd.read_csv('../input/bureau.csv')
bureau_balance=pd.read_csv('../input/bureau_balance.csv')
credit_card_balance=pd.read_csv('../input/credit_card_balance.csv')
installments_payments=pd.read_csv('../input/installments_payments.csv')
pos_cash_balance=pd.read_csv('../input/POS_CASH_balance.csv')
previous_application=pd.read_csv('../input/previous_application.csv')
print('the data has beens loaded!')

**2. some basic description of the files**


In [11]:
file_list={ 'application_train': application_train, 
           'bureau': bureau, 
           'bureau_balance': bureau_balance,
           'credit_card_balance': credit_card_balance,
           'installments_payments': installments_payments , 
           'pos_cash_balance': pos_cash_balance,
           'previous_application': previous_application}
for file_name in file_list.keys():
    print('the %s has %d rows , %d columns and %d features' %( file_name, file_list[file_name].shape[0], file_list[file_name].shape[1],len(file_list[file_name].columns.values)))
    

**3. EDA part**



**3.1 missing data**

In [14]:
def count_missing_data(file_name):
    file_values=file_list[file_name]
    count=file_values.isnull().sum()
    total=file_values.isnull().count()
    percentage=count/total *100
    df=pd.concat([count,percentage],axis=1,keys=['count','percentage'])
    return df
for file_keys in file_list:
    df=count_missing_data(file_keys)
    print('The percentage of missing data of features in %s :\n %r \n'
          %(file_keys, df.sort_values(by=['percentage'],ascending=False).head(10)))

**3.2 target**

In [12]:
target_1=np.sum(application_train['TARGET']==1)
target_0=np.sum(application_train['TARGET']==0)
plt.figure()
plt.pie([target_0,target_1],explode=[0,0.25],labels=['0','1'],shadow=True,autopct='%0.2f %%')
plt.show()

**3.3 categorical features**

OCCUPATION_TYPE

In [20]:
def plot_categorical_features(file,feature):
    '''
    use this function to plot categorical features
    '''
    plt.figure()
    plt.title('The bar chart of feature: '+ feature)
    data=file[feature].value_counts()
    labels=data.index
    x=list(range(len(data)))
    plt.bar(x,data.values)
    plt.xticks(x,labels,rotation='vertical')
    plt.ylabel('COUNT')
    
    
plot_categorical_features(application_train,'OCCUPATION_TYPE')

NAME_EDUCATION_TYPE

In [21]:
plot_categorical_features(application_train,'NAME_EDUCATION_TYPE')

NAME_INCOME_TYPE

In [22]:
plot_categorical_features(application_train,'NAME_INCOME_TYPE')

**3.4 Numerical features**

CREDICT_AMOUNT

In [ ]:
def plot_numberical_features(file,feature,bins=50):
    data=file[feature]
    plt.figure()
    plt.title('The distribution of feature: ' + feature)
    sns.distplot(data.dropna(),rug=True,bins=bins)
    plt.show()
    
plot_numberical_features(application_train,'AMT_CREDIT')

**3.5 correlation**

In [3]:
def plot_correlation_map(file):
    data=file.corr()  #return type is dataframe
    plt.figure()
    sns.heatmap(data.values,annot=False)
    plt.show()
    
plot_correlation_map(application_train)


**4 preprocessing**

**4.1 dealing with missing data**

In [39]:
def missing_features(file_name,percentage):
    df=count_missing_data(file_name)
    feature_list=df.index
    drop_list=list(feature_list[df['percentage']>percentage])
    print('The number of features whose percentage of missing values are greater than {}% in {} is {}'.format(percentage,file_name,len(drop_list)))
    return drop_list

drop_list = missing_features('application_train',20)
#application_train=application_train.drop(drop_list, axis = 1)
print('droping finished')
print('the size of application_train is now {}'.format(application_train.shape))

**4.2 imputing missing data**

NUMERICAL FEATURES

In [48]:
def impute_numerical_features(file, strategy='median'):
    numerical_list=[]
    for feature in file.columns.values:
        if not file[feature].dtype == 'object':
            numerical_list.append(feature)
    file[numerical_list]=MICEImputer(initial_strategy=strategy, n_imputations = 50, 
                                      n_nearest_features = 20, verbose = True).fit_transform(file[numerical_list])
            
            
impute_numerical_features(application_train)

**4.3 Encoding Labels**

In [40]:
def encoding_label(file):
    for features in file.columns.values:
        if file[features].dtype=='object':
            file[features]=file[features]
            le=LabelEncoder()
            file[features]=le.fit_transform(file[features])
    print('The file has been label encoded')
    
    
encoding_label(application_train)